In [1]:
import numpy
import bempp.api
from bempp.api.operators.boundary import sparse, laplace
from scipy.sparse.linalg import gmres

In [2]:
def rhs_fun(x, n, domain_index,result):
    global phi_static
    result[:] = phi_static
    
def generate_grid(filename):
    
    face = open(filename+'.face','r').read()
    vert = open(filename+'.vert','r').read()

    faces = numpy.vstack(numpy.char.split(face.split('\n')[0:-1]))[:,:3].astype(int) - 1
    verts = numpy.vstack(numpy.char.split(vert.split('\n')[0:-1]))[:,:3].astype(float)

    grid = bempp.api.grid_from_element_data(verts.transpose(), faces.transpose())
    
    N = grid.leaf_view.entity_count(0)
    elements = list(grid.leaf_view.entity_iterator(0))
    area = numpy.zeros(N)

    # remove zero areas
    for i in range(N):
        area[i] = elements[i].geometry.volume

    area_nonzero = numpy.where(area>1e-12)[0]
    
    faces_nonzero = faces[area_nonzero,:]
    
    grid = bempp.api.grid_from_element_data(verts.transpose(), faces_nonzero.transpose())
    
    return grid

def energy_mol(mesh_diel, mesh_stern, phi_static, eps_s):

    grid_diel = generate_grid(mesh_diel)
    grid_stern = generate_grid(mesh_stern)

    space_diel = bempp.api.function_space(grid_diel, "DP", 0)
    space_stern = bempp.api.function_space(grid_stern, "DP", 0)
    
    N_diel = grid_diel.leaf_view.entity_count(0)
    N_stern = grid_stern.leaf_view.entity_count(0)
    
    elements_d = list(grid_diel.leaf_view.entity_iterator(0))
    elements_s = list(grid_stern.leaf_view.entity_iterator(0))

    phis_grid_fun = bempp.api.GridFunction(space_diel, fun=rhs_fun)

    rhs = numpy.concatenate([eps_s*phis_grid_fun.coefficients, 
                      numpy.zeros(N_stern)])
    

    M11   = laplace.single_layer(space_diel, space_diel, space_diel) 
    M12   = laplace.single_layer(space_stern, space_diel, space_diel) 
    M21   = laplace.single_layer(space_diel, space_stern, space_stern)
    M22   = laplace.single_layer(space_stern, space_stern, space_stern) 

    blocked = bempp.api.BlockedOperator(2, 2)
    blocked[0,0] = M11
    blocked[0,1] = M12
    blocked[1,0] = M21
    blocked[1,1] = M22
    op_discrete = blocked.strong_form()
    
    sigma, info = gmres(op_discrete, rhs, tol=1e-5, maxiter=500, restart = 1000)
    
    if info>0:
        print 'Not converged, %i iterations'%info
    elif info<0:
        print 'Solver diverges'
    
    sigma_d = sigma[:N_diel]
    sigma_s = sigma[N_diel:]

    
    qe = 1.60217662e-19
    Rw = 1.4
    Na = 6.0221409e23
    eps_0 = 8.854187817e-12
    
    #water_charge = 0.83*qe/(numpy.pi*Rw**2) * Na*qe*1e10/(1000*eps_0)
    water_dipole = 6.17e-30/(1e-10)/((2*Rw)**2) * Na*qe*1e10/(1000*eps_0)
    print water_dipole
    print sigma_s[:5]*1.4
    print max(abs(sigma_s*1.4))
    '''
    test_charge = numpy.where(abs(sigma_d)*1.4>water_dipole)[0]
    #print abs(sigma_d)
    if len(test_charge)>0:
        print 'Unphysical charge in dielectric surface'
        print len(test_charge)
    '''    
    
    test_charge = numpy.where(abs(sigma_s)*1.4>water_dipole)[0]
    if len(test_charge)>0:
        print 'Unphysical charge in stern surface'
        print len(test_charge)
    
    area_d = numpy.zeros(N_diel)
    for i in range(N_diel):
        area_d[i] = elements_d[i].geometry.volume
    
    energy = 0.5*numpy.sum(area_d*sigma_d*phi_static)

    return energy, float(len(test_charge))/N_stern

In [3]:
phi_static = 2.5 #kcal/mol/e
phi_static *= 4.184 # kJ/mol/e

qe = 1.60217662e-19
Na = 6.0221409e23
eps_0 = 8.854187817e-12
eps_s = 80.

conv_factor = (1000*eps_0)/(qe**2*Na*1e10)

problem_folder = 'mobley_test/'
namefile = 'actually_all_names.txt'
num_lines = sum(1 for line in open(problem_folder+namefile))
energy = numpy.zeros(num_lines)
test_charge_arr = numpy.zeros(num_lines)
it = 0
for line in file(problem_folder+namefile):
    mol = line[:-1]

    mesh_stern = problem_folder+mol+'/surf_d02_stern'
    mesh_diel = problem_folder+mol+'/surf_d02'

    try: 
        e, test_charge = energy_mol(mesh_diel, mesh_stern, phi_static, eps_s)
    except:
        print 'Something went wrong'
        e = 0
        test_charge = 0.
            
    ekJ = e*conv_factor # kJ/mol
    ekcal = ekJ/4.184   # kcal/mol
    
    energy[it] = ekcal
    test_charge_arr[it] = test_charge
    
    print 'Run %i for '%it + mol + '. Energy = %1.3f kcal/mol'%energy[it]
    
    it += 1
    
numpy.savetxt('energy_bem.txt', energy)
numpy.savetxt('test_charge.txt', test_charge_arr)

857.593881087
[  429.12036468 -6463.34071732 -1924.84140545 -2293.26327865 -2306.02682132]
8961.9051181
Unphysical charge in stern surface
236
Run 0 for 1112_tetrachloroethane. Energy = 8.332 kcal/mol
857.593881087
[-1888.42409692 -7413.76724079 -3415.81789373 -2126.92090869 -2628.15560428]
8750.8958934
Unphysical charge in stern surface
228
Run 1 for 111_trichloroethane. Energy = 7.710 kcal/mol
857.593881087
[  123.10488383 -1359.9159103   -435.86581843   588.41557785   421.14802158]
16856.032268
Unphysical charge in stern surface
397
Run 2 for 111_trifluoro_222_trimethoxyethane. Energy = 10.014 kcal/mol
857.593881087
[-1211.0051211  -7307.24616667 -3437.8949957  -2349.04455012 -2764.80981279]
7658.00357789
Unphysical charge in stern surface
246
Run 3 for 111_trifluoropropan_2_ol. Energy = 7.552 kcal/mol
857.593881087
[ 468.02983523 -396.77729807 -412.79843638  481.2895933  -205.10258785]
8835.92540532
Unphysical charge in stern surface
379
Run 4 for 111_trimethoxyethane. Energy = 9.4

857.593881087
[ -79.65122926  -78.76323633  107.95629758  150.77350838 -165.12648622]
16606.1204527
Unphysical charge in stern surface
454
Run 41 for 1_bromoheptane. Energy = 11.807 kcal/mol
857.593881087
[ -303.37261662 -7432.55580325 -7649.88320103  -393.861546   -1338.40245979]
9339.87295373
Unphysical charge in stern surface
402
Run 42 for 1_bromohexane. Energy = 10.746 kcal/mol
857.593881087
[-1225.16538401 -1929.56273156 -2363.9487735   -475.22500229  -972.74812916]
14431.7211104
Unphysical charge in stern surface
506
Run 43 for 1_bromooctane. Energy = 12.914 kcal/mol
857.593881087
[-1147.56592912 -1939.47018291 -1373.24166554 -1498.32389418 -1529.13917718]
8634.36054386
Unphysical charge in stern surface
334
Run 44 for 1_bromopentane. Energy = 9.675 kcal/mol
857.593881087
[-1091.75865148 -1113.36175994  -668.22034668 -1033.1549642   -552.59931623]
12494.7125101
Unphysical charge in stern surface
223
Run 45 for 1_bromopropane. Energy = 7.464 kcal/mol
857.593881087
[-1533.771798  

857.593881087
[ 1622.62498141 -1690.21698174   406.93374372   761.62482816   139.01941523]
21370.9745525
Unphysical charge in stern surface
347
Run 83 for 24_dimethylpyridine. Energy = 9.408 kcal/mol
857.593881087
[   19.87519307  -406.90933659    54.81219626   804.09858293 -1387.02903858]
251890.618464
Unphysical charge in stern surface
395
Run 84 for 25_dimethylphenol. Energy = 9.908 kcal/mol
857.593881087
[-1036.91656504 -1822.20205303 -1734.33789425 -1123.59779913 -1722.54482558]
20522.1048344
Unphysical charge in stern surface
346
Run 85 for 25_dimethylpyridine. Energy = 9.430 kcal/mol
857.593881087
[-1642.20115043 -1556.0169315  -1875.85357675  -842.62593343  -920.73294379]
12414.2949269
Unphysical charge in stern surface
341
Run 86 for 25_dimethyltetrahydrofuran. Energy = 9.200 kcal/mol
857.593881087
[-113.13503224  -72.12684934  -30.86203399 -458.20425718 -335.97481756]
138004.549152
Unphysical charge in stern surface
394
Run 87 for 26_dimethylaniline. Energy = 9.910 kcal/mol
8

857.593881087
[ -996.63321161 -5288.12005515 -5643.21145741 -1391.2166995   -734.24858588]
34051.0707794
Unphysical charge in stern surface
329
Run 125 for 2_methylpentane. Energy = 9.407 kcal/mol
857.593881087
[  645.41401155 -5751.20750232 -2486.77138429 -2050.41787775 -2981.62356799]
10611.2514281
Unphysical charge in stern surface
247
Run 126 for 2_methylpropan_1_ol. Energy = 7.650 kcal/mol
857.593881087
[-1062.42750901 -5410.90743128 -2559.45168645 -2669.88405779 -2598.73104587]
22367.354641
Unphysical charge in stern surface
252
Run 127 for 2_methylpropan_2_ol. Energy = 7.682 kcal/mol
857.593881087
[-2231.16487128 -3263.11583253 -2215.73828314 -2597.51744302 -6253.52119469]
26172.8622426
Unphysical charge in stern surface
236
Run 128 for 2_methylpropane. Energy = 7.398 kcal/mol
857.593881087
[-2693.96351015 -5437.96378301 -2214.8773832   1396.96598056 -2333.75053741]
33773.2069984
Unphysical charge in stern surface
292
Run 129 for 2_methylpropene. Energy = 7.133 kcal/mol
857.5938

857.593881087
[ -992.50171079 -4061.2932113  -2105.45695589 -2504.29034119 -2036.49987595]
68154.2225829
Unphysical charge in stern surface
285
Run 167 for 3_methylbutanoic_acid. Energy = 8.658 kcal/mol
857.593881087
[-2258.12616312 -3188.42693747 -2077.48994329 -2720.36542173 -5682.63168829]
13235.9322598
Unphysical charge in stern surface
447
Run 168 for 3_methylheptane. Energy = 11.416 kcal/mol
857.593881087
[ -828.0404861  -1079.6358285  -1170.02705419 -1427.7283853  -1471.82026273]
149836.644735
Unphysical charge in stern surface
394
Run 169 for 3_methylhexane. Energy = 10.169 kcal/mol
857.593881087
[ -700.51621026 -1811.73875924  -653.17736565 -2254.73725362 -1604.75816   ]
211011.832415
Unphysical charge in stern surface
336
Run 170 for 3_methylpentane. Energy = 9.225 kcal/mol
857.593881087
[ 1592.41970282 -1879.82948219  -634.77786772   538.46567245  -346.83098396]
18977.3718058
Unphysical charge in stern surface
269
Run 171 for 3_methylpyridine. Energy = 8.286 kcal/mol
857.593

...

857.593881087
[-2830.12800869 -5167.54790884 -3101.81883873 -3497.4486303  -2808.83331949]
5167.54790884
Unphysical charge in stern surface
137
Run 294 for diiodomethane. Energy = 6.858 kcal/mol
857.593881087
[-224.60508441 -268.64445356 -468.01195907 -458.22843199  -87.58659455]
54462.1602868
Unphysical charge in stern surface
423
Run 295 for diisopropyl_ether. Energy = 9.895 kcal/mol
857.593881087
[ -998.43944922  1211.89374205 -1678.71554391 -3627.0405737  -2192.77541191]
29730.5901487
Unphysical charge in stern surface
374
Run 296 for diisopropylamine. Energy = 9.816 kcal/mol
857.593881087
[  630.84005903 -2464.37551594  -310.66737452  1802.47213696   -13.67716514]
7947.4735017
Unphysical charge in stern surface
248
Run 297 for dimethoxymethane. Energy = 7.060 kcal/mol
857.593881087
[-3020.95142016 -2982.59787243 -2547.99086023  -196.79398856 -2981.00195627]
33876.6946443
Unphysical charge in stern surface
201
Run 298 for dimethyl_disulfide. Energy = 7.373 kcal/mol
857.593881087
[-

857.593881087
[-1313.15164915 -2296.23408158  -798.5535732  -2182.29990516 -2771.83573925]
9405.87857097
Unphysical charge in stern surface
376
Run 337 for hexan_3_ol. Energy = 9.837 kcal/mol
857.593881087
[-1918.30902149 -5506.70147082 -2145.88287909 -2518.73333365 -3451.64277396]
7894.69485765
Unphysical charge in stern surface
352
Run 338 for hexanal. Energy = 9.802 kcal/mol
857.593881087
[-2375.8929379  -3148.78633887 -2477.88565564 -2177.24616446 -2447.76485206]
8148.88292243
Unphysical charge in stern surface
373
Run 339 for hexanoic_acid. Energy = 10.201 kcal/mol
857.593881087
[-964.6841941  -845.75347188   45.68092294  220.11696729 -891.78783812]
1160.85543294
Unphysical charge in stern surface
12
Run 340 for hydrazine. Energy = 3.930 kcal/mol
Something went wrong
Run 341 for hydrogen_sulfide. Energy = 0.000 kcal/mol
857.593881087
[-1000.01412047 -1911.94578762 -1661.75143242 -2125.98454043 -2196.06679815]
11277.6283663
Unphysical charge in stern surface
111
Run 342 for imidazo

857.593881087
[ 1506.56176377 -1536.26606041   697.52226709 -1043.98850226   466.95940509]
13260.4981136
Unphysical charge in stern surface
302
Run 380 for methyl_methanesulfonate. Energy = 7.624 kcal/mol
857.593881087
[-1465.34289662 -5447.32570089 -2639.48417598 -2252.53812752 -2762.08981005]
103864.783442
Unphysical charge in stern surface
552
Run 381 for methyl_octanoate. Energy = 13.628 kcal/mol
857.593881087
[-2432.34309626 -2991.00263793 -2093.08115056 -1789.50786178 -2897.37275136]
159049.595383
Unphysical charge in stern surface
449
Run 382 for methyl_p_methoxybenzoate. Energy = 11.545 kcal/mol
857.593881087
[-2798.02736509 -3264.6309616  -1033.76253551 -3630.04373342 -1566.26975568]
179260.405606
Unphysical charge in stern surface
426
Run 383 for methyl_p_nitrobenzoate. Energy = 11.383 kcal/mol
857.593881087
[-106229.35146542  100251.22308493    -975.0611719    -5967.67174354
   -2158.09937057]
106229.351465
Unphysical charge in stern surface
380
Run 384 for methyl_pentanoate

857.593881087
[ -5014.58017866 -12711.32863721  -2114.68966582  -2961.61218705
  -1087.88809849]
39897.7943783
Unphysical charge in stern surface
226
Run 422 for n_propylamine. Energy = 6.950 kcal/mol
857.593881087
[-3175.94517825 -2147.55449467 -2702.98572088 -1830.04682758 -2878.00286625]
134355.190841
Unphysical charge in stern surface
441
Run 423 for n_propylbenzene. Energy = 10.672 kcal/mol
857.593881087
[-1526.35726883 -6046.40736087 -1629.99896135 -1963.08651119 -1582.3740457 ]
13016.2150851
Unphysical charge in stern surface
403
Run 424 for n_propylcyclopentane. Energy = 10.578 kcal/mol
857.593881087
[ 3726.26418393 -2107.96317291   139.49887375  -536.09016424   356.29419576]
143621.808885
Unphysical charge in stern surface
402
Run 425 for naphthalene. Energy = 9.884 kcal/mol
857.593881087
[-2833.57143826 -2175.78699374 -2563.03145005 -1891.00229389 -3073.63030307]
185377.463647
Unphysical charge in stern surface
305
Run 426 for nitrobenzene. Energy = 8.737 kcal/mol
857.5938810

857.593881087
[-1034.75917551 -2258.22909667 -1434.55892451 -2255.94988778 -1428.98431602]
37544.7098721
Unphysical charge in stern surface
175
Run 465 for prop_2_en_1_ol. Energy = 6.394 kcal/mol
857.593881087
[-495.51730121  -96.64176034 -526.04146723  279.54749768 -432.37370956]
29264.1254086
Unphysical charge in stern surface
215
Run 466 for propan_1_ol. Energy = 6.813 kcal/mol
857.593881087
[-1694.82970913 -4457.67209518 -2058.76320375 -2324.75280842 -1901.57050701]
49521.6872623
Unphysical charge in stern surface
231
Run 467 for propan_2_ol. Energy = 6.775 kcal/mol
857.593881087
[-1322.34934416 -1689.30874058 -1327.94577075 -1496.82325938 -1381.74632025]
97148.7641464
Unphysical charge in stern surface
189
Run 468 for propane. Energy = 6.498 kcal/mol
857.593881087
[ -964.33348082  1467.30635193 -2464.7391943  -3549.61696456 -2133.49277844]
6164.72234198
Unphysical charge in stern surface
167
Run 469 for propanenitrile. Energy = 6.621 kcal/mol
857.593881087
[ -355.49056412  -747.26

In [12]:
mesh_stern = 'mobley_test/nitroethane/surf_d02_stern'
mesh_diel = 'mobley_test/nitroethane/surf_d02'

qe = 1.60217662e-19
Na = 6.0221409e23
eps_0 = 8.854187817e-12
eps_s = 5.5#80.

phi_static = 46. #kJ/mol/e
conv_factor = (1000*eps_0)/(qe**2*Na*1e10)


e, test_charge = energy_mol(mesh_diel, mesh_stern, phi_static, eps_s)
ekj = e * conv_factor#/ ((qe**2*Na*1e10)/(1000*eps_0))
ekcal = ekj/4.184
#print e, e / ((qe**2*Na*1e10)/(1000*eps_0)), e/(1000*4.184) #kJ/mol
print ekcal

857.593881087
[-278.6805626  -496.77562928 -318.64588094 -114.92787056 -278.07517599]
28648.1780082
Unphysical charge in stern surface
45
8.82394356584


In [33]:
grid_diel = generate_grid(mesh_diel)
grid_stern = generate_grid(mesh_stern)

space_diel = bempp.api.function_space(grid_diel, "DP", 0)
space_stern = bempp.api.function_space(grid_stern, "DP", 0)
    
N_diel = grid_diel.leaf_view.entity_count(0)
N_stern = grid_stern.leaf_view.entity_count(0)
    
elements_d = list(grid_diel.leaf_view.entity_iterator(0))
elements_s = list(grid_stern.leaf_view.entity_iterator(0))

print elements_d[0].geometry.corners
print elements_d[0].geometry.local2global(numpy.array([[1.,0.,0.]]))

[[-0.802 -0.885 -0.46 ]
 [ 1.119  0.807  1.406]
 [ 0.339  0.758  0.614]]


AttributeError: 'Geometry' object has no attribute 'global2local'

In [20]:
F = 1000*eps_0/(qe**2*Na*1e10)
1/(4*numpy.pi*F*4.184)
9081457.48434/(332.063*4*numpy.pi)
(qe**2*Na*1e10)/(eps_0)
2353.39653991/40

58.834913497749994

In [ ]:
1.54